In [1]:
import pandas as pd
import geopandas as gpd
import mytools
import math,simplekml,os

# ---------------制作线条图层-----------------

## 导入和导出设置 

In [2]:
out_file = 'G:/1-规划/图层/全省区域规划图层20200110/'

In [3]:
data = gpd.read_file('G:/1-规划/图层/全省区域规划图层20200110/全省区域规划图层20200110.TAB',encoding='gbk')

In [4]:
data['区域类型'] = data['区域类型'].replace({'3A景区':'景区','4A景区':'景区','5A景区':'景区'})

In [5]:
use_columns = ['地市名称', '县市名称', '区域名称', '区域类型', '覆盖面积', 'geometry']

In [6]:
data_use = data[use_columns]

## 设置变量

In [7]:
data_use.groupby('区域类型').count()

,地市名称,县市名称,区域名称,覆盖面积,geometry
区域类型,,,,,
一般城区,25,25,25,25,25
主城区,16,16,16,16,16
乡镇,1193,1193,1193,1193,1193
县城,72,72,72,72,72
景区,186,186,186,186,186
热点,85,85,85,85,85


In [8]:
#- 变量 - 设置经纬度和参与循环的字段
lon,lat,use_id = ['lon','lat','区域名称']

In [9]:
#- 颜色 - 按照字符设置颜色
colour_data = '区域类型'
colour_dic = {'主城区':'red','一般城区':'orange','县城':'yellow','乡镇':'blue','景区':'green','热点':'white'}

In [10]:
#- 标注 -添加标注列
de_col = use_columns
de_col.remove('geometry')

In [11]:
#- 视野 - 在1400*900的分辨率下  view = 70米对应：125公里（差不多能看到武汉主城区）  700对应1200公里（差不多能看到湖北省）
view = 50000

## 数据整理

In [12]:
data_use.head(1)

,地市名称,县市名称,区域名称,区域类型,覆盖面积,geometry
0,孝感,安陆,接官乡,乡镇,1.02963,"POLYGON ((113.75599 31.43954, 113.75366 31.438..."


In [13]:
#添加标注列
data_use['description']=''
for inde_1, name_1 in enumerate(de_col):
    data_use['linshi']=de_col[inde_1]+' : '+data_use[de_col[inde_1]].astype('str')+'\n'
    data_use['description'] = data_use['description']+data_use['linshi']
    data_use.drop(columns='linshi')

In [14]:
#多边形的质心
data_use[['lon','lat']] = data_use.apply(lambda x:pd.Series(x['geometry'].centroid.coords[:][0]),axis=1)#

In [15]:
data_sectors= data_use.copy()
#添加lod的polygon
data_sectors['lod_dis']= view
data_sectors['lon1'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(45* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat1'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(45* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['lon2'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(225* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat2'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(225* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['list_data']=[([lon_1, lat_1,lon_2, lat_2]) 
 for lon_1, lat_1,lon_2, lat_2 in zip(data_sectors['lon1'],data_sectors['lat1'],data_sectors['lon2'],data_sectors['lat2'])]

In [16]:
# df = data_sectors.copy()
# #添加颜色列-数字
# df = data_sectors.copy()
# df['colour']=1
# df['colour'] = pd.cut(df['colour'],bins=[-0.1, 0.7, 0.936, 1.1],labels =['red', 'yellow', 'green'])
# df['colour'] = df['colour'].astype('object')
# # df.loc[df[rsrp_count]<100,'colour']='white'

In [17]:
df = data_sectors.copy()
#添加颜色列-文字
df['colour']='white'
for name_col in colour_dic:
    df.loc[df[colour_data]==name_col,'colour'] =colour_dic[name_col]

In [18]:
#最终使用的列
df_use = df.reindex(columns=[use_id,colour_data, 'list_data','colour','description','geometry'])   

## 生成图层

In [19]:
def make_kml(data_t,name='红色栅格',cc='ff0000ff',xiankuan=0,namea='grid_id',list_data = 'list_data',description='description',minlodpixels=1):
    print(data_t.shape,cc,xiankuan,namea,list_data,description,minlodpixels)
    style = simplekml.Style()
    style.linestyle.color = simplekml.Color.changealphaint(220, cc)  # 最终线条上色
    style.linestyle.width = xiankuan
#     style.polystyle.color = simplekml.Color.changealphaint(135, cc )  # 最终形状上色
    lod1 = simplekml.Lod(minlodpixels=minlodpixels, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
    grid_red = kml.newfolder(name=name)
    for grid,list_data ,description_str,geo in zip(data_t[namea],data_t[list_data],data_t[description],data_t['geometry']):
#         pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list_data[0])#栅格使用
        pol_r = grid_red.newlinestring(name=grid,coords=list(geo.exterior.coords))
        pol_r.description = description_str
        pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
        lon_dd,lat_dd,lon1_dd,lat1_dd = list_data
        latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                              minaltitude=None, maxaltitude=None, altitudemode=None)
        pol_r.region.latlonaltbox = latlonaltbox
        pol_r.region.lod = lod1
        pol_r.style=style

In [20]:
df_use_p = df_use.explode()

In [21]:
df_use_p = df_use_p.reset_index(drop=True)

In [ ]:
colour_dic = {'主城区':'red','一般城区':'orange','县城':'yellow','乡镇':'blue','景区':'green','热点':'white'}

In [27]:
kml = simplekml.Kml()
for name_c,data_t in df_use_p.groupby('colour'):#red:ff0000ff,green:ff008000,yellow:ff00ffff,white:ffffffff,bule:ffff0000
    if 'red' ==name_c:
        make_kml(data_t,name='主城区{}个'.format(data_t.shape[0]),cc='ff0000ff',namea=use_id,minlodpixels=50,xiankuan=2)
    elif 'blue' == name_c:
        make_kml(data_t,name='乡镇{}个'.format(data_t.shape[0]),cc='ffff0000',namea=use_id,minlodpixels=50,xiankuan=2)
    elif 'yellow' == name_c:
        make_kml(data_t,name='县城{}个'.format(data_t.shape[0]),cc='ff00ffff',namea=use_id,minlodpixels=50,xiankuan=2)
    elif 'white' == name_c:
        make_kml(data_t,name='热点{}个'.format(data_t.shape[0]),cc='ffffffff',namea=use_id,minlodpixels=50,xiankuan=2)
    elif 'green'  == name_c:
        make_kml(data_t,name='景区{}个'.format(data_t.shape[0]),cc='ff008000',namea=use_id,minlodpixels=50,xiankuan=2)
    elif 'orange'  == name_c:
        make_kml(data_t,name='一般城区{}个'.format(data_t.shape[0]),cc='ff00a5ff',namea=use_id,minlodpixels=50,xiankuan=2)
kml.savekmz('{}输出线条格式.kml'.format(out_file), False)
print('生成图层成功存放在{}'.format(out_file))

(1205, 6) ffff0000 2 区域名称 list_data description 50
(194, 6) ff008000 2 区域名称 list_data description 50
(57, 6) ff00a5ff 2 区域名称 list_data description 50
(16, 6) ff0000ff 2 区域名称 list_data description 50
(85, 6) ffffffff 2 区域名称 list_data description 50
(79, 6) ff00ffff 2 区域名称 list_data description 50
生成图层成功存放在G:/1-规划/图层/全省区域规划图层20200110/
